<h1>뉴욕타임스 기사 학습 후 텍스트생성</h1>

In [29]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
import numpy as np 
from keras.utils import to_categorical

In [58]:
import pandas as pd

In [63]:
df=pd.read_csv("nyt.csv")
# df.head()['webURL']
df.info()
df.head(1)['snippet']


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1324 entries, 0 to 1323
Data columns (total 15 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   articleID         1324 non-null   object
 1   articleWordCount  1324 non-null   int64 
 2   byline            1324 non-null   object
 3   documentType      1324 non-null   object
 4   headline          1324 non-null   object
 5   keywords          1324 non-null   object
 6   multimedia        1324 non-null   int64 
 7   newDesk           1324 non-null   object
 8   printPage         1324 non-null   int64 
 9   pubDate           1324 non-null   object
 10  sectionName       1324 non-null   object
 11  snippet           1324 non-null   object
 12  source            1324 non-null   object
 13  typeOfMaterial    1324 non-null   object
 14  webURL            1324 non-null   object
dtypes: int64(3), object(12)
memory usage: 155.3+ KB


0       Former N.F.L. Cheerleaders’ Settlement Offer: ...
1       E.P.A. to Unveil a New Rule. Its Effect: Less ...
2                                 The New Noma, Explained
3                                                 Unknown
4                                                 Unknown
                              ...                        
1319    This Common Question Reinforces the Gender Pay...
1320                                   Anna, Llama and Me
1321           Gen. Michael Hayden Has One Regret: Russia
1322                         There Is Nothin’ Like a Tune
1323                                              Unknown
Name: headline, Length: 1324, dtype: object

In [65]:
df['headline'].isnull().sum()
df['headline'].isnull().values.any()


False

In [ ]:
df.headline.values
list(df.headline.values)

In [69]:
headline = []
headline.extend(list(df.headline.values))
headline[:10]

['Former N.F.L. Cheerleaders’ Settlement Offer: $1 and a Meeting With Goodell',
 'E.P.A. to Unveil a New Rule. Its Effect: Less Science in Policymaking.',
 'The New Noma, Explained',
 'Unknown',
 'Unknown',
 'Unknown',
 'Unknown',
 'Unknown',
 'How a Bag of Texas Dirt  Became a Times Tradition',
 'Is School a Place for Self-Expression?']

In [70]:
len(headline) #1324개 = unknown포함 

1324

In [73]:
headline=[n for n in headline if  n != 'Unknown'] # n이 Unknown하고 같이 않을 때 제목으로 뽑겠다 
len(headline) #1214개 = unknown 불포함 

1214

In [74]:
headline[:5]

['Former N.F.L. Cheerleaders’ Settlement Offer: $1 and a Meeting With Goodell',
 'E.P.A. to Unveil a New Rule. Its Effect: Less Science in Policymaking.',
 'The New Noma, Explained',
 'How a Bag of Texas Dirt  Became a Times Tradition',
 'Is School a Place for Self-Expression?']

In [78]:
from string import punctuation
print(punctuation) # !"#$%&'()*+,-./:;<=>?@[\]^_`{|}~

!"#$%&'()*+,-./:;<=>?@[\]^_`{|}~


In [84]:
s = 'former nfl cheerleaders’ settlement offer 1 and a meeting with goodel'
for c in s:
    print(c)
    print('='*50)

f
o
r
m
e
r
 
n
f
l
 
c
h
e
e
r
l
e
a
d
e
r
s
’
 
s
e
t
t
l
e
m
e
n
t
 
o
f
f
e
r
 
1
 
a
n
d
 
a
 
m
e
e
t
i
n
g
 
w
i
t
h
 
g
o
o
d
e
l


In [89]:
# "’" <-처리가 안되었음 유니코드이기 때문에 코드를 변환해줘야함.
"’".encode('utf8').decode('ascii', 'ignore')

''

In [94]:
##함수 생성
def repreprocessing(s):
    s=s.encode('utf8').decode('ascii', 'ignore') # ’를 지우기 위해
    return ''.join(c for c in s if c not in punctuation).lower() # c는 뉴스기사 
    # print(s) # headlines' names
    # print("="*50)
    #s에 전달된 뉴스기사 제목에 대해 전처리(점 제거, 소문자)
    #그런데 ' <-가 제거가 안되었음. 따로 제거 해야함.
    
text=[repreprocessing(x) for x in headline ]
#               <-2->           <- 1 ->
text[:5]
# text

['former nfl cheerleaders settlement offer 1 and a meeting with goodell',
 'epa to unveil a new rule its effect less science in policymaking',
 'the new noma explained',
 'how a bag of texas dirt  became a times tradition',
 'is school a place for selfexpression']

In [95]:
t = Tokenizer()
t.fit_on_texts(text)

In [100]:
t.word_index
print(len(t.word_index)) # 3493개 단어 
print(len(text)) #1214개 뉴스기사 헤드라인


3493
1214


In [98]:
vocabSize=len(t.word_index)+1 # 3493개 +1

In [106]:
sequences = []
for line in text: # 1214개 헤드라인 , 3493개 단어 
    encoded=t.texts_to_sequences([line])[0] # 
    for i in range(1, len(encoded)):
        sequence=encoded[:i+1]
        sequences.append(sequence)
sequences[:11]


[[99, 269],
 [99, 269, 371],
 [99, 269, 371, 1115],
 [99, 269, 371, 1115, 582],
 [99, 269, 371, 1115, 582, 52],
 [99, 269, 371, 1115, 582, 52, 7],
 [99, 269, 371, 1115, 582, 52, 7, 2],
 [99, 269, 371, 1115, 582, 52, 7, 2, 372],
 [99, 269, 371, 1115, 582, 52, 7, 2, 372, 10],
 [99, 269, 371, 1115, 582, 52, 7, 2, 372, 10, 1116],
 [100, 3]]

In [114]:
t.word_counts # 단어들의 등장 횟수 
t.word_index # 단어들의 인덱스

indextoWord = {} # indextoWord에 v인덱스에 해당하는 k단어를 저장할것이다.
for k, v in t.word_index.items():
    indextoWord[v] = k # {1:'the}

In [115]:
print(indextoWord[300])

dept


In [117]:
maxLen=max(len(s) for s in sequences)
print(maxLen) # 가장긴 것 24 

24


In [118]:
sequences=pad_sequences(sequences, maxlen=maxLen, padding='pre')
print(sequences[:5])

[[   0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0   99  269]
 [   0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0   99  269  371]
 [   0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0   99  269  371 1115]
 [   0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0   99  269  371 1115  582]
 [   0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0   99  269  371 1115  582   52]]


In [119]:
x = sequences[:,:-1]
y = sequences[:,-1]

In [120]:
print(x[0])
print(y[0])

[ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0 99]
269


In [121]:
#ex) 1=010000000000, 11=000000000001
y = to_categorical(y, num_classes=vocabSize)
y

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 1., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)

In [122]:
y.shape # (7803, 3494)

(7803, 3494)

In [48]:
##모델생성
from keras.models import Sequential
from keras.layers import Embedding, Dense, SimpleRNN, LSTM

In [123]:
##모델 생성
model = Sequential()
model.add(Embedding(vocabSize, 10, input_length=maxLen-1))
#입력데이터의 길이는 maxLen에서 y를 뺀 maxLen-1이 된다. 
model.add(LSTM(128))
model.add(Dense(vocabSize, activation='softmax'))
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])
model.fit(x, y, epochs=200)

Epoch 1/200
244/244 [==============================] - 3s 12ms/step - loss: 7.6565 - accuracy: 0.0288
Epoch 2/200
244/244 [==============================] - 3s 13ms/step - loss: 7.1263 - accuracy: 0.0309
Epoch 3/200
244/244 [==============================] - 3s 13ms/step - loss: 6.9902 - accuracy: 0.0322
Epoch 4/200
244/244 [==============================] - 3s 13ms/step - loss: 6.8649 - accuracy: 0.0413
Epoch 5/200
244/244 [==============================] - 3s 13ms/step - loss: 6.7223 - accuracy: 0.0436
Epoch 6/200
244/244 [==============================] - 3s 12ms/step - loss: 6.5675 - accuracy: 0.0470
Epoch 7/200
244/244 [==============================] - 3s 13ms/step - loss: 6.4238 - accuracy: 0.0483
Epoch 8/200
244/244 [==============================] - 3s 12ms/step - loss: 6.2705 - accuracy: 0.0528
Epoch 9/200
244/244 [==============================] - 3s 13ms/step - loss: 6.0386 - accuracy: 0.0583
Epoch 10/200
244/244 [==============================] - 3s 13ms/step - loss: 5.862

In [131]:
def sentence_generation(model, t, cw, n):
    initWord = cw # 시작단어 
    sentence = ''
    for _ in range(n): # 10번 반복한다  _ :10번 반복 하는 동안 값을 받는 변수가 없다. 
        encoded=t.texts_to_sequences([cw])[0] # [2]
        encoded=pad_sequences([encoded], maxlen=23, padding='pre')
        # print(encoded) #[2] ->-[0 0 0 0 2]
        result = model.predict_classes(encoded)
        # print(result) #[3] ==word_index[3]의 index
        for word, index in t.word_index.items():
            if index==result:
                break
        cw=cw+" "+word 
        sentence = sentence+" "+word
    initWord+sentence
    return sentence

sentence_generation(model, t, 'i', 10) 
# t: Tokenizer
#'i' :시작단어
# 단어 10개를 예측 해라

' want to be rich and im not sorry allies wait'